## Pandas

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
p = os.path.join(os.path.dirname('__file__'), '..')
sys.path.append(p)
from common import *
DATA_DIR = '../data/'

### Series

In [ ]:
# A one-dimensional labeled array capable of holding any data type:
# integers, strings, floats, Python objects

In [ ]:
s = pd.Series([1,2,3])                         #auto generate index (by default 0,1,2...)
s = pd.Series([1,2,3], index=['A','B','C'])   #manually assign index for each row
s['A']

In [ ]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
population

### DataFrame

In [ ]:
'''
A 2-dimensional labeled data structure with columns of potentially different types. 

Inputs:
    Dict of 1D lists, dicts, or Series
    2-D numpy.ndarray
    A Series
    Another DataFrame
'''

In [ ]:
# From dictionary of lists (or Series)
data = {'one' : [1., 2., 3.],
        'two' : [1., 2., 3.]}
pd.DataFrame(data)

In [ ]:
# From Numpy Array
pd.DataFrame(np.random.rand(3, 2),
             columns=['foo', 'bar'],
             index=['a', 'b', 'c'])

### DataFrame Index

Pandas objects are designed to facilitate operations such as joins across datasets, which depend on many aspects of set arithmetic.

In [ ]:
indA = pd.Index([1, 3, 5, 7, 9])
indB = pd.Index([2, 3, 5, 7, 11])

In [ ]:
indA & indB  # intersection
indA | indB  # union
indA.values

### Load File

In [ ]:
# For .read_csv, always use header=0 when you know row 0 is the header row
pd.read_csv(DATA_DIR + 'advertising.csv', header=0).head()

In [ ]:
# read CSV file directly from a URL and save the results
url_data = pd.read_csv('http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv', index_col=0)

In [ ]:
# Save to file
url_data.to_csv(DATA_DIR + 'advertising.csv', index=False)

### Viewing Data

In [ ]:
df = pd.read_csv(DATA_DIR + 'pandastutorial.csv', header=0).head()
df.head(2)

In [ ]:
df.info()      #Metadata about table
df.columns
df.values
df.head(2)
df.tail(2)
df.shape       # count of (rows, columns)

In [ ]:
# Sort Values
df.sort_values(by='Age').head(2)

### Data Types

In [ ]:
df.Age = df.Age.astype('float16')
df.Age

### Select by Index

In [ ]:
# iloc lets you do numpy like indexing
df.columns.get_loc("Age")     #Get column index by name

In [ ]:
df.iloc[0]                    #Get row by index
df[0:3]                       #Get rows 0-2
df.iloc[:,2]                  #Get all column 2 data
df.iloc[3,3]                  #Get value in cell (1,1)

### Select by Name

In [ ]:
df['Survived']                  #Get column by name
df['Age'][0:3]                  #Select first 3 rows in Age column
df[['Age','Sex']][0:3]          #Select first 3 rows in Age and Sex columns

### Updating Tables

In [ ]:
df2 = df.copy()                           #Create copy
df2["NewColumn"] = "DefaultValue"         #Create new column
df2['Sex'] = "Unknown"                    #Override all values in column
df2.iat[0,0] = 9999                       #Set value by index (row,col)
df2.at[3,'Pclass'] = 7777                 #Set value in row 2, col 'Pclass'
df2[df2.SibSp == 1] = "OVERRIDE"          #Set all values in rows with 1 sibling

# Set column matching query
df.loc[(df.Age.isnull()) & (df.Sex=="female") & (df.Pclass==1),'Age'] = 9.9

### Basic Statistics

In [ ]:
# Only works on numerical columns
# Ignores missing values
df.describe()     #Summary stats

In [ ]:
# Mean for all rows in a column
df.mean()         

In [ ]:
# Mean for all cells in a row (non-null)
df.mean(axis=1)        

In [ ]:
df.sum()          # Sum for all non-null values in a column
df.count()        # Count non-null values in each column

In [ ]:
df['Embarked'].unique()    #Get unique values
df['Embarked'].nunique()   #Get count of unique, non-null values
df['Embarked'].mode()[0]   #Mode is weird, you need to extract the value

### Filtering

In [ ]:
# Get rows that match condition
res = df[df.Age > 10]                       

In [ ]:
# Same, but return only 2 columns and 3 rows
df[df.Age > 10][['Sex', 'Age']][:3]         

In [ ]:
#Get rows where Embarked is S or C
df[df.Embarked.isin(['S','C'])][:3]

### Querying

In [ ]:
# Query and select specific columns
df.loc[df.Age > 10, ['Age', 'Sex']]

In [ ]:
# Simple query
df[ (df['Sex'] == 'male') & (df['Pclass'] == 1) ]     #Return males in Pclass 1

In [ ]:
# Complex Queries

#Create new column equal to query
df['FancyName'] = np.where(
    (
        (df.Name.str.contains("Master.")) 
        | (df.Name.str.contains("Rev.")) 
        | (df.Name.str.contains("Dr."))
        | (df.Name.str.contains("Dr.")) 
        | (df.Name.str.contains("Sir."))
    ),
    True,
    False
)

#Get all rows equal to query
upper_class_men = df.loc[ 
    (df.Sex.str.lower() == "male") & (
        (df.Name.str.contains("Master.")) 
        | (df.Name.str.contains("Rev.")) 
        | (df.Name.str.contains("Dr."))
        | (df.Name.str.contains("Dr.")) 
        | (df.Name.str.contains("Sir.")) 
    )
][['Fare','Age','Survived']]
upper_class_men.mean()

### Missing Values

In [ ]:
# Pandas uses the value np.nan to represent missing data
# These methods return a copy of the DataFrame without modifying it

In [ ]:
df3 = df.copy()                               #Just to be extra safe ;)
df3.isnull().sum()                            #Show # of null values in each Column
df3[df3.Age.isnull()][['Name','Age']]         #Show rows where Age is null
df3.isnull()                                  #Return True/False if cell is null
df3.fillna(value={"Cabin":"DEFAULT_CABIN"})   #Set null values in column to default value 
df3.dropna(how='any')                         #Drop rows with missing data in ANY cell
df3.fillna(value="DEFAULT").head(2)           #Fill cells missing data w 5

### Data Cleaning

In [ ]:
df4 = df.copy()

In [ ]:
# New column
df4['SurvivalPct'] = df4['Survived'] / df4['PassengerId']
df4['Gender'] = df4['Sex'].map( lambda x: x[0].upper() )    #New column Gender with M or F
df4['Gender'] = df4['Sex'].map( {'male':1,'female':0} )     #Update Gender values to 0 or 1

In [ ]:
#Drop Row
df4 = df4.drop(0)
df4[['Name','Sex','Age']].head(3)

In [ ]:
#Drop Column
df4 = df4.drop('Age',1)         # 1 means Column axis
df4.head(2)

### Group By

In [ ]:
#For each column, return sum of all rows that equal 'female'
df.groupby('Sex').sum()            

In [ ]:
# Group by multiple columns
df.groupby(['Sex','Embarked']).mean()      

In [ ]:
#Get the mean fare price for each Embarked category
df.groupby('Embarked').Fare.mean()         

In [ ]:
# Query, Then Group
df[df['Sex'] == 'female'].groupby('Embarked').Survived.sum()

### Histogram

In [ ]:
import pylab

# Pandas auto-generates buckets
df['Age'].hist()
pylab.show()

# Set specific buckets
df['Age'].dropna().hist(bins=16, range=(0,80), alpha=.5)
pylab.show()

### Plots

In [ ]:
import matplotlib.pyplot as plt

sex_group = df[['Sex','Age']].groupby('Sex')
sex_group.size()

total_passengers = sex_group.mean()
myplot = total_passengers.plot(kind='bar', title="Mean Age by Sex")
myplot.set_xlabel("Sex")
myplot.set_ylabel("Age")
plt.show()

### Pivot Tables

* https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html

In [ ]:
#http://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html
'''
1. Pivot on Age (the value we perform calculations on)
2. Group by Sex and Embarked
3. Include statistics: sum, mean, and count
4. Show totals at bottom!
''';

In [ ]:
pd.pivot_table(df, values='Age', index=['Sex', 'Embarked'], 
               aggfunc=[np.sum, np.mean, np.count_nonzero], 
               margins=True)

### Aggregations


* count()	Total number of items
* first(), last()	First and last item
* mean(), median()	Mean and median
* min(), max()	Minimum and maximum
* std(), var()	Standard deviation and variance
* mad()	Mean absolute deviation
* prod()	Product of all items
* sum()	Sum of all items

In [ ]:
#Simple Aggregation
df.groupby('Embarked').aggregate({'Survived':np.sum, 'Age':np.mean})

In [ ]:
#Complex Aggregation
aggregations = {
    'Survived': { # work on the "Survived" column
        'total_passengers': 'count',  # get count, and call this result 'total_passengers'
        'total_survived': 'sum', # get sum, call result 'total_survived'
        'mean_survival': 'mean',
        'survival_percent': lambda x: float(sum(x)) / len(x)
    },
    'Fare': {     # Now work on the "Fare" column
        'max_fare': 'max',   # Find the max, call the result "max_fare"
        'min_fare': 'min',
        'total_of_all_fares': 'sum',
        'fare_range': lambda x: max(x) - min(x)  # Calculate the fare range per group
    },
    'Age': ["count", "max"]  # Calculate two results for 'Age' column
}
 
df.groupby('Sex').agg(aggregations)

### Iterate

In [ ]:
# Iterate through rows
for idx,row in df.iterrows():
    print(idx)

In [ ]:
# Iterate through columns
for name, val in df.iteritems():
    print(val)

### Concat

In [ ]:
class display():
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

In [ ]:
# Signature in Pandas v0.18
# pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
#          keys=None, levels=None, names=None, verify_integrity=False,
#          copy=True)

In [ ]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

In [ ]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])

# Stack Rows
display('df1', 'df2', 'pd.concat([df1, df2])')

In [ ]:
# Stack Columns
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [1, 2])
display('df1', 'df2', 'pd.concat([df1, df2], axis=1)')

In [ ]:
# Verify no overlapping indices
try:
    pd.concat([df1, df2], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

In [ ]:
# Ignore the existing index (create new!)
pd.concat([df1, df2], ignore_index=True)

In [ ]:
# Join with different column names
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

In [ ]:
# Inner join (Only include shared columns)
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

### Append data

In [ ]:
# Append dataframe - same as pd.concat([df1,df2])
df1.append(df2)     #returns new object (not in-place)

### Merge/Join

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

In [ ]:
# Join on employee column
df3 = pd.merge(df1, df2, on='employee')
df3

In [ ]:
# Join on column when left and right have different names
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

In [ ]:
# Join on Index column
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

In [ ]:
# Join on Index (shorthand)
df1a.join(df2a)

In [ ]:
# Join on Index and Column Name
pd.merge(df1a, df3, left_index=True, right_on='name')

In [ ]:
# Many-to-one 
# Two key columns contains duplicate entries
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

In [ ]:
# Many-to-many
# Both left and right arrays contain duplicates
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

In [ ]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])

In [ ]:
# Inner Join (intersection of overlapping names)
display('df6', 'df7', 'pd.merge(df6, df7)')

In [ ]:
# Outer Join (Union of names)
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

In [ ]:
# Left Join (take all names in left and only include matching names in right table)
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

### Strings

* https://jakevdp.github.io/PythonDataScienceHandbook/03.10-working-with-strings.html

### Dates

* https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html

In [ ]:
from datetime import datetime
datetime(year=2015, month=7, day=4)

In [ ]:
from dateutil import parser
date = parser.parse("4th of July, 2015")
date

In [ ]:
# Create a sequence of dates
date = np.array('2015-07-05', dtype=np.datetime64)
mydates = np.arange(12) + date
mydates

In [ ]:
# Add time delta to date
date = pd.to_datetime("4th of July, 2015")
date + pd.to_timedelta(np.arange(12), 'D')

In [ ]:
# Convert date strings and python datetimes to Pandas dates
dates = pd.to_datetime([datetime(2015, 7, 3), '4th of July, 2015',
                       '2015-Jul-6', '07-07-2015', '20150708'])
dates

In [ ]:
# Create a date range
pd.date_range('2015-07-03', '2015-07-10')

In [ ]:
# Create a daterange of n periods with f frequency
pd.date_range('2015-07-03', periods=8, freq='6H')

### Query by dates

In [ ]:
# Query Dates
mydates[mydates > np.datetime64('2015-07-11')]

In [ ]:
index = pd.DatetimeIndex(['2014-07-04', '2014-08-04',
                          '2015-07-04', '2015-08-04'])
data = pd.Series([0, 1, 2, 3], index=index)
data

In [ ]:
# Grab date slice
data['2014-07-04':'2015-07-04']

In [ ]:
data['2015']

### Resample time-series interval

In [ ]:
from pandas_datareader import data

goog = data.DataReader('GOOG', start='2010', end='2016',
                       data_source='google')
goog.head()

* This can be done using the resample() method, or the much simpler asfreq() method. 
* resample() is fundamentally a data aggregation
* asfreq() is fundamentally a data selection.

In [ ]:
goog.plot(alpha=0.5, style='-')
goog.resample('BA').mean().plot(style=':')
goog.asfreq('BA').plot(style='--');

In [ ]:
# Fill in missing time-series data
data.asfreq('D').plot(ax=ax[0], marker='o')

data.asfreq('D', method='bfill').plot(ax=ax[1], style='-o')
data.asfreq('D', method='ffill').plot(ax=ax[1], style='--o')
ax[1].legend(["back-fill", "forward-fill"]);